## Data: Online Retail II

This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers.

The dataset can be accessed at the following link: https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci/

Attribute Information:

- InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.
- UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).
- CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal. The name of the country where a customer resides.

## Business Understanding

Market Basket Analysis is a data mining technique employed to discover relationships and patterns within large datasets, particularly in the context of market analysis. By identifying frequently co-occurring items in transactions, businesses can gain valuable insights into customer behavior, optimize product placement, and enhance overall marketing strategies.

## Objectives

1. **Association Rule Discovery**:
    Identify associations and correlations among products or items in a dataset. Discover rules that indicate the likelihood of certain items being bought together.
2. **Cross-Selling Opportunities**:
    Uncover opportunities for cross-selling by understanding which products are frequently purchased together.
3. **Promotion Planning**:
    Optimize promotional campaigns by identifying items that are frequently bought together. Design effective promotions and discounts to incentivize the purchase of complementary products.
4. **Optimizing Product Layout**:
    Arrange products in-store or online in a way that encourages the purchase of related items, creating a more convenient and satisfying shopping experience.

## Key Metrics

- **Support** 
    - Support measures how frequently an association rule happens in a dataset.
- **Confidence** 
    - Confidence measures how strong an association rule is. 
    - That is to say, in market basket analysis terms, how likely is a second product to be present in the basket if the first is.
    - Confidence(A→B) = Support(A∪B)/Support(A)×100%
    - Confidence(B→A) = Support(A∪B)/Support(B)×100%
- **Lift**
    - Lift measures how much more likely two items are to be bought together compared to being bought individually at random.
    - Lift(A→B)= Support(A∪B)/Support(A)×Support(B)
    - If Lift = 1, it means there is no association between A and B.
    - If Lift > 1, it indicates that A and B are more likely to be bought together than randomly.
    - If Lift < 1, it suggests that A and B are less likely to be bought together than randomly.
- **Leverage**
    - Leverage measures the difference between the observed frequency of A and B occurring together and the frequency that would be expected if A and B were statistically independent.
    - Leverage(A→B)=Support(A∪B)−(Support(A)×Support(B))
    - Positive leverage indicates that the items appear together more frequently than expected by chance.
    - Zero leverage means the items occur together exactly as expected based on their individual supports.
    - Negative leverage implies the items co-occur less frequently than expected.
    
- **Conviction**
    - Conviction measures the ratio of the expected frequency that A occurs without B to the frequency that A occurs when B is present.
    - Conviction(A→B)= 1−Support(B)/1−Confidence(A→B)
    - Conviction(B→A)= 1−Support(A)/1−Confidence(B→A)
    - If Conviction = 1, it means that A and B are independent of each other.
    - If Conviction > 1, it suggests that the presence of B has increased the likelihood of A, indicating a strong association.
    - If Conviction < 1, it indicates a negative association between A and B.
- **Zhang’s metric**
    - Measure that looks not only at positive associations, but also negative. It can says, for instances, if buying A makes someone NOT buy B.
    - Value: < 0 if there’s a negative association (dissociation), > 0 if there’s a positive association where -1 and 1 are the extreme values.

## Import Modules

In [5]:
import os
import re

import pandas as pd
import numpy as np

import plotly.express as px
import networkx as nx

import json
import copy

In [6]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [7]:
from networks.RulesGraphManager import RulesGraphManager as RGM
from networks.ProductNetwork import ProductNetwork
from networks.CrossSellingProducts import CrossSellingProducts

from grouper.NxGrouper import NxGrouper
from charts.HeatmapXTab import HeatmapCrosstab

from echarts.EgraphForce import EgraphForce
from echarts.EgraphStandard import EgraphStandard
from echarts.JupyterEcharts import JupyterEcharts

In [8]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

## Data Preparation

### Load Dataset

In [9]:
pathname = os.path.join("F:\\Data\\datas", "online_retail_II.csv")
df = pd.read_csv(
    pathname, 
    dtype = {'Customer ID': str, 'Invoice': str},
    parse_dates = ['InvoiceDate']
)

In [10]:
COL_CUSTOMER_ID = 'Customer ID'
COL_ORDER_DATE = 'InvoiceDate'
COL_ORDER_ID = 'Invoice'
COL_VALUE = 'Sales'
COL_QUANTITY = 'Quantity'

In [16]:
df.shape

(1067371, 8)

### Data Cleaning

#### Drop missing values

In [15]:
df.isna().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [21]:
df = df.dropna()

#### Ignore duplicate rows

There are two possible reasons for duplicate rows. 
1. The cashier might have scanned each product individually instead of scanning once and entering the quantity, resulting in multiple entries for each product on the invoice. 
2. The cashier might have accidentally scanned the same product twice. 

**Decision process**: The first scenario is more common, so we assume that the duplicate rows are not mistakes. <br>
**Action**: We don't address duplicates because we lack clarification on the matter. 

In [22]:
df.loc[df.duplicated()].shape

(26479, 8)

#### Drop rows with StockCode column has "TEST" value

In [60]:
df = df[~df['StockCode'].str.contains('TEST')]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country


#### Drop rows with price has less or equal to zero

In [51]:
df = df[~df['Price'] <= 0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
4674,489825,22076,6 RIBBONS EMPIRE,12,2009-12-02 13:34:00,0.0,16126.0,United Kingdom
6781,489998,48185,DOOR MAT FAIRY CAKE,2,2009-12-03 11:19:00,0.0,15658.0,United Kingdom
16107,490727,M,Manual,1,2009-12-07 16:38:00,0.0,17231.0,United Kingdom
18738,490961,22065,CHRISTMAS PUDDING TRINKET POT,1,2009-12-08 15:25:00,0.0,14108.0,United Kingdom
18739,490961,22142,CHRISTMAS CRAFT WHITE FAIRY,12,2009-12-08 15:25:00,0.0,14108.0,United Kingdom
32916,492079,85042,ANTIQUE LILY FAIRY LIGHTS,8,2009-12-15 13:49:00,0.0,15070.0,United Kingdom
40101,492760,21143,ANTIQUE GLASS HEART DECORATION,12,2009-12-18 14:22:00,0.0,18071.0,United Kingdom
47126,493761,79320,FLAMINGO LIGHTS,24,2010-01-06 14:54:00,0.0,14258.0,United Kingdom
48342,493899,22355,"CHARLOTTE BAG , SUKI DESIGN",10,2010-01-08 10:43:00,0.0,12417.0,Belgium
57619,494607,21533,RETRO SPOT LARGE MILK JUG,12,2010-01-15 12:43:00,0.0,16858.0,United Kingdom


In [43]:
df[df['Quantity'] <= 0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
...,...,...,...,...,...,...,...,...
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
1067002,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [61]:
df[df['StockCode'].str.isalpha()]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
89,489439,POST,POSTAGE,3,2009-12-01 09:28:00,18.00,12682.0,France
126,489444,POST,POSTAGE,1,2009-12-01 09:55:00,141.00,12636.0,USA
173,489447,POST,POSTAGE,1,2009-12-01 10:10:00,130.00,12362.0,Belgium
625,489526,POST,POSTAGE,6,2009-12-01 11:50:00,18.00,12533.0,Germany
735,C489535,D,Discount,-1,2009-12-01 12:11:00,9.00,15299.0,United Kingdom
...,...,...,...,...,...,...,...,...
1067002,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
1067191,581570,POST,POSTAGE,1,2011-12-09 11:59:00,18.00,12662.0,Germany
1067228,581574,POST,POSTAGE,2,2011-12-09 12:09:00,18.00,12526.0,Germany
1067229,581578,POST,POSTAGE,3,2011-12-09 12:16:00,18.00,12713.0,Germany


In [65]:
df[df['StockCode'].str.isalpha()]['StockCode'].value_counts()

StockCode
POST      2019
M         1115
D          174
ADJUST      61
PADS        19
DOT         16
CRUK        16
Name: count, dtype: int64

#### Drop cancelation invoice rows

**Action**: drop rows

In [67]:
df = df[~df['Invoice'].str.startswith('C')]

#### Trim space in description column

In [72]:
df['Description'] = df['Description'].str.strip()
df['Description'] = df['Description'].replace(r'\s{2,}', ' ', regex=True)

### Get Sales Column

In [74]:
df['Sales'] = df['Price'] * df['Quantity']

### Data Selection

In [81]:
df_basket = df[['Description', 'Invoice']]

### Data Encoding

In [75]:
def encoding_data(df, COL_ITEM_ID, COL_ORDER_ID):
    df = df.copy()
    df[COL_ITEM_ID] = df[COL_ITEM_ID].transform(lambda x: [x])
    df = df.groupby(COL_ORDER_ID, as_index=False).sum()[COL_ITEM_ID]
    df = df.map(lambda x: list(set(x)))
                          
    encoder = TransactionEncoder()
    one_hot_transactions = pd.DataFrame(
        encoder.fit(df).transform(df), columns=encoder.columns_
    )
    
    return one_hot_transactions

In [82]:
df_transactions = encoding_data(df_basket, 'Description', 'Invoice')

## Data Mining (Rules Extraction)

The following parameters are configured for the algorithm:

- Maximum Combination Length
    - We set the maximum combination length to 2 items.
    - This choice is made to focus on pairs of items, allowing for a more targeted analysis of co-occurrences.

- Minimum Co-Occurrence Support Threshold
    - A minimum co-occurrence support threshold of 1% is established to filter out infrequent itemsets. 
    - This ensures that only associations with a significant presence in the dataset are considered.

In [100]:
frequent_itemsets = apriori(df_transactions, min_support= 0.01, use_colnames=True, max_len = 2, low_memory=True)
rules = association_rules(frequent_itemsets, metric="support", min_threshold = 0.01)

In [103]:
rules['antecedents'] = rules['antecedents'].map(lambda x: ''.join(list(x)))
rules['consequents'] = rules['consequents'].map(lambda x: ''.join(list(x)))

In [101]:
rules.shape

(354, 10)

In [113]:
rules.sort_values('support', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
308,RED HANGING HEART T-LIGHT HOLDER,WHITE HANGING HEART T-LIGHT HOLDER,0.044125,0.132240,0.031193,0.706928,5.345783,0.025358,2.960912,0.850464
309,WHITE HANGING HEART T-LIGHT HOLDER,RED HANGING HEART T-LIGHT HOLDER,0.132240,0.044125,0.031193,0.235884,5.345783,0.025358,1.250955,0.936822
351,WOODEN PICTURE FRAME WHITE FINISH,WOODEN FRAME ANTIQUE WHITE,0.044856,0.047967,0.026865,0.598914,12.485996,0.024713,2.373640,0.963111
350,WOODEN FRAME ANTIQUE WHITE,WOODEN PICTURE FRAME WHITE FINISH,0.047967,0.044856,0.026865,0.560068,12.485996,0.024713,2.171117,0.966259
247,LUNCH BAG SUKI DESIGN,LUNCH BAG SPACEBOY DESIGN,0.053351,0.050699,0.024024,0.450304,8.881855,0.021319,1.726957,0.937423
...,...,...,...,...,...,...,...,...,...,...
88,GIN + TONIC DIET METAL SIGN,NO SINGING METAL SIGN,0.037470,0.026242,0.010064,0.268592,10.235019,0.009081,1.331347,0.937421
136,JUMBO BAG OWLS,JUMBO BAG RED RETROSPOT,0.024890,0.070665,0.010064,0.404348,5.722017,0.008305,1.560197,0.846301
137,JUMBO BAG RED RETROSPOT,JUMBO BAG OWLS,0.070665,0.024890,0.010064,0.142420,5.722017,0.008305,1.137048,0.887986
287,PLASTERS IN TIN SPACEBOY,PLASTERS IN TIN SKULLS,0.031275,0.024511,0.010037,0.320934,13.093480,0.009270,1.436516,0.953445


## Network Visualization with Echarts

### Network Profile

We profile the network to get a quick summary of our products network 

In [104]:
myRGM = RGM(rules, 'antecedents', 'consequents')
df_nodes, df_edges = myRGM.get_graph_features()

In [105]:
df_nodes_profile = NxGrouper.greedy_modularity_communities(df_nodes, df_edges, 4)

In [106]:
force = EgraphForce(
    df_edges, 
    df_nodes_profile, 
    col_source='antecedents', 
    col_target='consequents', 
    col_name='nodes',
)
profile_force_option = force.get_option()

In [107]:
# JupyterEcharts.show(profile_force_option)

![Description](images/profile_network.png)

### Product Network

After we know our network profile, we can select and assess individual product networks.

In [108]:
MyPN = ProductNetwork(rules)
df_bfs, rules_bfs = MyPN.get_bfs_rules(['RED HANGING HEART T-LIGHT HOLDER'], 'support', 0, 3, 5)
df_nodes, df_edges = MyPN.get_graph_features(df_bfs, rules_bfs, strict_rules=True)

In [111]:
df_nodes['label'] = df_nodes['rank'].map(lambda x : {"show": True, "position": "right", "formatter": f"{x}"})

In [112]:
product_force = EgraphForce(
    df_edges, 
    df_nodes,
    col_category='depth',
    col_source='antecedents', 
    col_target='consequents',
    col_name='nodes',
)
product_force_option = product_force.get_option(show_legend=True)

In [ ]:
# JupyterEcharts.show(product_force_option)

![Description](images/product_network.png)

### Cross Selling Products

Using this technique, we can bundle our products that are frequently purchased together.

In [114]:
MyCSP = CrossSellingProducts(rules)
cross_selling_rules = MyCSP.get_cross_selling_products(max_support_ratio_diff=2.0, min_confidence=0.35)

In [115]:
df_nodes, df_edges = MyCSP.get_graph_features(cross_selling_rules)
df_nodes = NxGrouper.greedy_modularity_communities(df_nodes, df_edges)

In [116]:
csp_force = EgraphForce(
    df_edges, df_nodes,
    col_source='antecedents',
    col_target='consequents',
    col_name='nodes',
)
csp_force_option = csp_force.get_option()

In [ ]:
# JupyterEcharts.show(csp_force_option)

![Description](images/cross_selling_products.png)

##  Product Placement with Plotly Heatmap

We can display our products side by side using Plotly heatmap

In [117]:
MyHM = HeatmapCrosstab(rules)

In [118]:
tabular = MyHM.get_tabular_data(
    ['WHITE HANGING HEART T-LIGHT HOLDER', 'RED HANGING HEART T-LIGHT HOLDER'], 
    'support',
    max_col=10
)

In [119]:
fig = MyHM.plot_heatmap(tabular)

In [121]:
# fig.show()

In [ ]:
# fig.write_image("images/heatmap_xtab.png")

![Description](images/heatmap_xtab.png)